In [240]:
import numpy as np
from scipy.stats import skewnorm
import pandas as pd

mean = [0, 0]
cov = [[1, 0], [0, 100]]  
np.random.multivariate_normal(mean, cov, 2)

array([[ -0.15417809, -15.18228154],
       [ -0.15069386,  26.232818  ]])

nu gaan we er iets mee doen

In [326]:
import numpy as np

np.random.seed(1234)
mu = [20,20] # center of distribution.
sigma_size_top, sigma_size_bot = np.random.uniform(5, 20, size=2)
cov_max = np.sqrt(sigma_size_top * sigma_size_bot) * 0.9 # Cov max can't be larger than sqrt of the other elements
sigma_cov = np.random.uniform(-cov_max, cov_max)
sigma = np.array([[sigma_size_top, sigma_cov],[sigma_cov, sigma_size_bot]])

display(sigma)
np.random.multivariate_normal(mu, sigma, 100)


array([[ 7.87279176, -1.1906369 ],
       [-1.1906369 , 14.33163157]])

array([[21.39591591, 22.81564765],
       [18.02366095, 16.00462615],
       [22.60323194, 19.18385066],
       [18.68780383, 27.52356085],
       [24.46504563, 17.42947179],
       [27.78774953, 21.95473949],
       [22.55161036, 19.27670292],
       [20.97314843, 17.50094653],
       [19.09643426, 22.17979355],
       [25.27468752, 16.57737804],
       [23.27914077, 18.02672238],
       [19.82372035, 24.04847151],
       [18.30802381, 24.96020079],
       [17.60656757, 21.28294624],
       [22.5848139 , 16.68589751],
       [22.1873377 , 19.98057994],
       [23.98991512, 24.25267284],
       [20.51402261, 17.79968534],
       [23.13376674, 13.21391154],
       [20.6953963 , 16.22479219],
       [24.62665738, 25.5939209 ],
       [22.94044195, 17.92074621],
       [26.55769596, 19.60288464],
       [16.99521076, 17.63805421],
       [18.26570274, 19.41591283],
       [18.25531155, 21.03556733],
       [27.28658316, 18.18324046],
       [19.19405809, 18.9051955 ],
       [22.52068383,